In [1]:
import pandas as pd
import numpy as np

import re
import pickle
from helpers import *
from gensim.utils import lemmatize
from nltk.stem import WordNetLemmatizer
# from prep_tweet_for_model import *


import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Run Predictions

In [2]:
# Load stopwords
stopwords2 = open(access_folder() + 'morestopwords.txt', "r").read().split('\n')
stopwords2 = list(filter(None, stopwords2)) # fastest
stopwords2 = [word for word in stopwords2 if word[0]!='#' ]
stopwords3 = open(access_folder() + 'common-english-prep-conj.txt' , "r").read().split(',')
stopwords4 = open(access_folder() + 'common-english-words.txt' , "r").read().split(',')
stopwords = stopwords2 +  stopwords3 + stopwords4

In [24]:
# Load data

# data = pd.read_csv('../data/final_merged_tweets.csv', index_col=0)

# Load old data set



cols_to_extract = [0,1,2,3,4,5,7,8,9,10,11,13,14]
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv', usecols=cols_to_extract)
tweets2 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_2.csv', usecols=cols_to_extract)
tweets3 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_3.csv', usecols=cols_to_extract)
tweets4 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_4.csv', usecols=cols_to_extract) 
tweets5 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_5.csv', usecols=cols_to_extract)
tweets6 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv', usecols=cols_to_extract)
tweets7 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_7.csv', usecols=cols_to_extract) 
tweets8 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_8.csv', usecols=cols_to_extract) 
tweets9 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_9.csv', usecols=cols_to_extract)

tweets = pd.concat([tweets1,tweets2,tweets3,tweets4,tweets5,tweets6,tweets7,tweets8,tweets9], axis=0)
del tweets1, tweets2, tweets3, tweets4, tweets5, tweets6, tweets7, tweets8, tweets9



#complete post_type column with normal tweet category
tweets.post_type.fillna('TWEET', inplace=True)

tweets['content'] = tweets.content.astype(str)

tweets[['external_author_id', 'author', 'region', 'language', 'post_type', 'account_type', 'retweet', 'account_category']] = \
tweets[['external_author_id', 'author', 'region', 'language', 'post_type', 'account_type', 'retweet', 'account_category']].astype('category')

tweets['publish_date'] = pd.to_datetime(tweets['publish_date'], format='%m/%d/%Y %H:%M')

data = tweets[(tweets.publish_date > '2016-10') & (tweets.publish_date < '2016-11-07')]


In [ ]:
# data['publish_date'] = pd.to_datetime(data['publish_date'])

In [ ]:
# data = data[(data.publish_date < '2015-08-01') & (data.publish_date > '2015-07-01')]

In [ ]:
data.shape

In [8]:
from __future__ import unicode_literals
import pandas as pd
import re
import numpy as np
import pickle
from helpers import *
from gensim.utils import lemmatize
from nltk.stem import WordNetLemmatizer
import operator
import sys


def remove_nonword(df):
    """Given a dataset with column called tweet_text, clean the tweets in tweet_text such that
    the hashtags are removed, the @ handles are removed, the RT (retweet) symbols are removed
    and the websites and punctuation are removed"""

    clean = df.replace('#\w+', '', regex=True).replace('@\w+', '', regex=True).replace('RT ', '', regex=True)
    clean.content = clean.content.apply(lambda x: re.sub(r'http\S+', '', x)).apply(lambda x: re.sub(r"'|\"|’|`|-|:|\?|~|,|\.", '', x))
    return clean


def token_stop(tweet, stopwords, filter_size=2):
    """Using functions from python file, tokenize and remove stopwords"""
    tokens = tokenizeRawTweetText(tweet)
    tokens = [token.lower() for token in tokens if token.lower() not in stopwords]
    return tokens



def lemmatize_tweets(tweet, min_length=3):
    """Takes in series that contains list of tokenizes words and lemmatizes them (gets rid of plurals etc)"""
    lemmatizer = WordNetLemmatizer()
    sentence = []
    for word in tweet:
        x = lemmatizer.lemmatize(word)
        if len(x) >= 3:
            sentence.append(x)
    return sentence


def create_array(lemmatized_tweet, total_words, dictionary):
    """Takes the lemmatized tweets and converts them to input array using the dictionary"""
    input_layer = np.zeros(total_words, dtype=float)
    for word in lemmatized_tweet:
        if word in dictionary.keys():
            input_layer[dictionary[word.lower()]] += 1
    return input_layer







#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Twokenize -- a tokenizer designed for Twitter text in English and some other European languages.
This tokenizer code has gone through a long history:
(1) Brendan O'Connor wrote original version in Python, http://github.com/brendano/tweetmotif
       TweetMotif: Exploratory Search and Topic Summarization for Twitter.
       Brendan O'Connor, Michel Krieger, and David Ahn.
       ICWSM-2010 (demo track), http://brenocon.com/oconnor_krieger_ahn.icwsm2010.tweetmotif.pdf
(2a) Kevin Gimpel and Daniel Mills modified it for POS tagging for the CMU ARK Twitter POS Tagger
(2b) Jason Baldridge and David Snyder ported it to Scala
(3) Brendan bugfixed the Scala port and merged with POS-specific changes
    for the CMU ARK Twitter POS Tagger
(4) Tobi Owoputi ported it back to Java and added many improvements (2012-06)
Current home is http://github.com/brendano/ark-tweet-nlp and http://www.ark.cs.cmu.edu/TweetNLP
There have been at least 2 other Java ports, but they are not in the lineage for the code here.
Ported to Python by Myle Ott <myleott@gmail.com>.
"""



try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass

def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'

Contractions = re.compile(u"(?i)(\w+)(n['’′]t|['’′]ve|['’′]ll|['’′]d|['’′]re|['’′]s|['’′]m)$", re.UNICODE)
Whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)

punctChars = r"['\"“”‘’.?!…,:;]"
#punctSeq   = punctChars+"+"	#'anthem'. => ' anthem '.
punctSeq   = r"['\"“”‘’]+|[.?!,…]+|[:;]+"	#'anthem'. => ' anthem ' .
entity     = r"&(?:amp|lt|gt|quot);"
#  URLs


# BTO 2012-06: everyone thinks the daringfireball regex should be better, but they're wrong.
# If you actually empirically test it the results are bad.
# Please see https://github.com/brendano/ark-tweet-nlp/pull/9

urlStart1  = r"(?:https?://|\bwww\.)"
commonTLDs = r"(?:com|org|edu|gov|net|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|pro|tel|travel|xxx)"
ccTLDs	 = r"(?:ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|" + \
r"bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|" + \
r"er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|" + \
r"hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|" + \
r"lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|" + \
r"nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|sk|" + \
r"sl|sm|sn|so|sr|ss|st|su|sv|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|" + \
r"va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|za|zm|zw)"	#TODO: remove obscure country domains?
urlStart2  = r"\b(?:[A-Za-z\d-])+(?:\.[A-Za-z0-9]+){0,3}\." + regex_or(commonTLDs, ccTLDs) + r"(?:\."+ccTLDs+r")?(?=\W|$)"
urlBody    = r"(?:[^\.\s<>][^\s<>]*?)?"
urlExtraCrapBeforeEnd = regex_or(punctChars, entity) + "+?"
urlEnd     = r"(?:\.\.+|[<>]|\s|$)"
url        = regex_or(urlStart1, urlStart2) + urlBody + "(?=(?:"+urlExtraCrapBeforeEnd+")?"+urlEnd+")"


# Numeric
timeLike   = r"\d+(?::\d+){1,2}"
#numNum     = r"\d+\.\d+"
numberWithCommas = r"(?:(?<!\d)\d{1,3},)+?\d{3}" + r"(?=(?:[^,\d]|$))"
numComb	 = u"[\u0024\u058f\u060b\u09f2\u09f3\u09fb\u0af1\u0bf9\u0e3f\u17db\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6\u00a2-\u00a5\u20a0-\u20b9]?\\d+(?:\\.\\d+)+%?"

# Abbreviations
boundaryNotDot = regex_or("$", r"\s", r"[“\"?!,:;]", entity)
aa1  = r"(?:[A-Za-z]\.){2,}(?=" + boundaryNotDot + ")"
aa2  = r"[^A-Za-z](?:[A-Za-z]\.){1,}[A-Za-z](?=" + boundaryNotDot + ")"
standardAbbreviations = r"\b(?:[Mm]r|[Mm]rs|[Mm]s|[Dd]r|[Ss]r|[Jj]r|[Rr]ep|[Ss]en|[Ss]t)\."
arbitraryAbbrev = regex_or(aa1, aa2, standardAbbreviations)
separators  = "(?:--+|―|—|~|–|=)"
decorations = u"(?:[♫♪]+|[★☆]+|[♥❤♡]+|[\u2639-\u263b]+|[\ue001-\uebbb]+)"
thingsThatSplitWords = r"[^\s\.,?\"]"
embeddedApostrophe = thingsThatSplitWords+r"+['’′]" + thingsThatSplitWords + "*"

#  Emoticons
# myleott: in Python the (?iu) flags affect the whole expression
#normalEyes = "(?iu)[:=]" # 8 and x are eyes but cause problems
normalEyes = "[:=]" # 8 and x are eyes but cause problems
wink = "[;]"
noseArea = "(?:|-|[^a-zA-Z0-9 ])" # doesn't get 😢
happyMouths = r"[D\)\]\}]+"
sadMouths = r"[\(\[\{]+"
tongue = "[pPd3]+"
otherMouths = r"(?:[oO]+|[/\\]+|[vV]+|[Ss]+|[|]+)" # remove forward slash if http://'s aren't cleaned

# mouth repetition examples:
# @aliciakeys Put it in a love song :-))
# @hellocalyclops =))=))=)) Oh well

# myleott: try to be as case insensitive as possible, but still not perfect, e.g., o.O fails
#bfLeft = u"(♥|0|o|°|v|\\$|t|x|;|\u0ca0|@|ʘ|•|・|◕|\\^|¬|\\*)".encode('utf-8')
bfLeft = u"(♥|0|[oO]|°|[vV]|\\$|[tT]|[xX]|;|\u0ca0|@|ʘ|•|・|◕|\\^|¬|\\*)"
bfCenter = r"(?:[\.]|[_-]+)"
bfRight = r"\2"
s3 = r"(?:--['\"])"
s4 = r"(?:<|&lt;|>|&gt;)[\._-]+(?:<|&lt;|>|&gt;)"
s5 = "(?:[.][_]+[.])"
# myleott: in Python the (?i) flag affects the whole expression
#basicface = "(?:(?i)" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5
basicface = "(?:" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5

eeLeft = r"[＼\\ƪԄ\(（<>;ヽ\-=~\*]+"
eeRight= u"[\\-=\\);'\u0022<>ʃ）/／ノﾉ丿╯σっµ~\\*]+"
eeSymbol = r"[^A-Za-z0-9\s\(\)\*:=-]"
eastEmote = eeLeft + "(?:"+basicface+"|" +eeSymbol+")+" + eeRight

oOEmote = r"(?:[oO]" + bfCenter + r"[oO])"


emoticon = regex_or(
        # Standard version  :) :( :] :D :P
        "(?:>|&gt;)?" + regex_or(normalEyes, wink) + regex_or(noseArea,"[Oo]") + regex_or(tongue+r"(?=\W|$|RT|rt|Rt)", otherMouths+r"(?=\W|$|RT|rt|Rt)", sadMouths, happyMouths),

        # reversed version (: D:  use positive lookbehind to remove "(word):"
        # because eyes on the right side is more ambiguous with the standard usage of : ;
        regex_or("(?<=(?: ))", "(?<=(?:^))") + regex_or(sadMouths,happyMouths,otherMouths) + noseArea + regex_or(normalEyes, wink) + "(?:<|&lt;)?",

        #inspired by http://en.wikipedia.org/wiki/User:Scapler/emoticons#East_Asian_style
        eastEmote.replace("2", "1", 1), basicface,
        # iOS 'emoji' characters (some smileys, some symbols) [\ue001-\uebbb]
        # TODO should try a big precompiled lexicon from Wikipedia, Dan Ramage told me (BTO) he does this

        # myleott: o.O and O.o are two of the biggest sources of differences
        #          between this and the Java version. One little hack won't hurt...
        oOEmote
)

Hearts = "(?:<+/?3+)+" #the other hearts are in decorations

Arrows = regex_or(r"(?:<*[-―—=]*>+|<+[-―—=]*>*)", u"[\u2190-\u21ff]+")

# BTO 2011-06: restored Hashtag, AtMention protection (dropped in original scala port) because it fixes
# "hello (#hashtag)" ==> "hello (#hashtag )"  WRONG
# "hello (#hashtag)" ==> "hello ( #hashtag )"  RIGHT
# "hello (@person)" ==> "hello (@person )"  WRONG
# "hello (@person)" ==> "hello ( @person )"  RIGHT
# ... Some sort of weird interaction with edgepunct I guess, because edgepunct
# has poor content-symbol detection.

# This also gets #1 #40 which probably aren't hashtags .. but good as tokens.
# If you want good hashtag identification, use a different regex.
Hashtag = "#[a-zA-Z0-9_]+"  #optional: lookbehind for \b
#optional: lookbehind for \b, max length 15
AtMention = "[@＠][a-zA-Z0-9_]+"

# I was worried this would conflict with at-mentions
# but seems ok in sample of 5800: 7 changes all email fixes
# http://www.regular-expressions.info/email.html
Bound = r"(?:\W|^|$)"
Email = regex_or("(?<=(?:\W))", "(?<=(?:^))") + r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}(?=" +Bound+")"

# We will be tokenizing using these regexps as delimiters
# Additionally, these things are "protected", meaning they shouldn't be further split themselves.
Protected  = re.compile(
    regex_or(
        Hearts,
        url,
        Email,
        timeLike,
        #numNum,
        numberWithCommas,
        numComb,
        emoticon,
        Arrows,
        entity,
        punctSeq,
        arbitraryAbbrev,
        separators,
        decorations,
        embeddedApostrophe,
        Hashtag,
        AtMention), re.UNICODE)

# Edge punctuation
# Want: 'foo' => ' foo '
# While also:   don't => don't
# the first is considered "edge punctuation".
# the second is word-internal punctuation -- don't want to mess with it.
# BTO (2011-06): the edgepunct system seems to be the #1 source of problems these days.
# I remember it causing lots of trouble in the past as well.  Would be good to revisit or eliminate.

# Note the 'smart quotes' (http://en.wikipedia.org/wiki/Smart_quotes)
#edgePunctChars    = r"'\"“”‘’«»{}\(\)\[\]\*&" #add \\p{So}? (symbols)
edgePunctChars    = u"'\"“”‘’«»{}\\(\\)\\[\\]\\*&" #add \\p{So}? (symbols)
edgePunct    = "[" + edgePunctChars + "]"
notEdgePunct = "[a-zA-Z0-9]" # content characters
offEdge = r"(^|$|:|;|\s|\.|,)"  # colon here gets "(hello):" ==> "( hello ):"
EdgePunctLeft  = re.compile(offEdge + "("+edgePunct+"+)("+notEdgePunct+")", re.UNICODE)
EdgePunctRight = re.compile("("+notEdgePunct+")("+edgePunct+"+)" + offEdge, re.UNICODE)

def splitEdgePunct(input):
    input = EdgePunctLeft.sub(r"\1\2 \3", input)
    input = EdgePunctRight.sub(r"\1 \2\3", input)
    return input

# The main work of tokenizing a tweet.
def simpleTokenize(text):

    # Do the no-brainers first
    splitPunctText = splitEdgePunct(text)

    textLength = len(splitPunctText)

    # BTO: the logic here got quite convoluted via the Scala porting detour
    # It would be good to switch back to a nice simple procedural style like in the Python version
    # ... Scala is such a pain.  Never again.

    # Find the matches for subsequences that should be protected,
    # e.g. URLs, 1.0, U.N.K.L.E., 12:53
    bads = []
    badSpans = []
    for match in Protected.finditer(splitPunctText):
        # The spans of the "bads" should not be split.
        if (match.start() != match.end()): #unnecessary?
            bads.append( [splitPunctText[match.start():match.end()]] )
            badSpans.append( (match.start(), match.end()) )

    # Create a list of indices to create the "goods", which can be
    # split. We are taking "bad" spans like
    #     List((2,5), (8,10))
    # to create
    #     List(0, 2, 5, 8, 10, 12)
    # where, e.g., "12" here would be the textLength
    # has an even length and no indices are the same
    indices = [0]
    for (first, second) in badSpans:
        indices.append(first)
        indices.append(second)
    indices.append(textLength)

    # Group the indices and map them to their respective portion of the string
    splitGoods = []
    for i in range(0, len(indices), 2):
        goodstr = splitPunctText[indices[i]:indices[i+1]]
        splitstr = goodstr.strip().split(" ")
        splitGoods.append(splitstr)

    #  Reinterpolate the 'good' and 'bad' Lists, ensuring that
    #  additonal tokens from last good item get included
    zippedStr = []
    for i in range(len(bads)):
        zippedStr = addAllnonempty(zippedStr, splitGoods[i])
        zippedStr = addAllnonempty(zippedStr, bads[i])
    zippedStr = addAllnonempty(zippedStr, splitGoods[len(bads)])

    # BTO: our POS tagger wants "ur" and "you're" to both be one token.
    # Uncomment to get "you 're"
    #splitStr = []
    #for tok in zippedStr:
    #    splitStr.extend(splitToken(tok))
    #zippedStr = splitStr

    return zippedStr

def addAllnonempty(master, smaller):
    for s in smaller:
        strim = s.strip()
        if (len(strim) > 0):
            master.append(strim)
    return master

# "foo   bar " => "foo bar"
def squeezeWhitespace(input):
    return Whitespace.sub(" ", input).strip()

# Final pass tokenization based on special patterns
def splitToken(token):
    m = Contractions.search(token)
    if m:
        return [m.group(1), m.group(2)]
    return [token]

# Assume 'text' has no HTML escaping.
def tokenize(text):
    return simpleTokenize(squeezeWhitespace(text))


# Twitter text comes HTML-escaped, so unescape it.
# We also first unescape &amp;'s, in case the text has been buggily double-escaped.
def normalizeTextForTagger(text):
    assert sys.version_info[0] >= 3 and sys.version_info[1] > 3, 'Python version >3.3 required'
    text = text.replace("&amp;", "&")
    text = html.unescape(text)
    return text

# This is intended for raw tweet text -- we do some HTML entity unescaping before running the tagger.
#
# This function normalizes the input text BEFORE calling the tokenizer.
# So the tokens you get back may not exactly correspond to
# substrings of the original text.
def tokenizeRawTweetText(text):
    tokens = tokenize(normalizeTextForTagger(text))
    return tokens


In [5]:
# First Clean
clean = remove_nonword(data)
del data

# load dictionary for model
dict_model = pickle.load(open('dictionary_nonews.pkl','rb'))  
total_words = len(dict_model)

In [9]:
# Tokenize and create input arrays
clean.content = clean.content.apply(lambda x: token_stop(x, stopwords)).apply(
                                                  lambda x: lemmatize_tweets(x, min_length=3)).apply(
                                                  lambda x: create_array(x, total_words, dict_model))


In [10]:
tokenizedtest = clean.content

In [11]:
tokenizedtest.shape

(171924,)

In [12]:
tokenizedtest.to_csv('inputsold2month.csv', header=None)
# data.to_csv('tweetsold2month', header=None)

In [13]:
# Parameters
learning_rate = 0.001
num_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
hidden_size1 = 6000
hidden_size2 = 1000
hidden_size3 = 100 # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 11        # Categories: graphics, sci.space and baseball

class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super().__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size1, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size1, hidden_size2, bias=True)
        self.layer_3 = nn.Linear(hidden_size2, hidden_size3, bias=True)
        self.output_layer = nn.Linear(hidden_size3, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out
    
net = OurNet(input_size, hidden_size1, hidden_size2, hidden_size3, num_classes)

net.load_state_dict(torch.load('./supervised_linear_nonews_traindataFINAL', map_location=lambda storage, loc: storage))




In [14]:
categories = ['Sports', 'Crime', 'Patriot', 'Entertainment', 
              'Anti-Trump', 'Fukushima', 'Foreign Countries',
              'Trump Support', 'Anti-Islam',
              'Black Support', 'Health']

# def predict_array(input_array, model, categories):
#     articles = Variable(torch.FloatTensor(input_array))
#     outputs = model(articles)
#     values, idx = torch.max(outputs, 0)
#     return categories[idx]

# predict_array(result, net, categories)



# Write Prediction Files

In [15]:
# file1 = open('prediction_scores_sep-nov','w') 
file2 = open('oldtweetspredictions.csv', 'w')
for idx, array in enumerate(tokenizedtest.values):
    articles= Variable(torch.FloatTensor(array))
    outputs = net(articles)
    values, idx1 = torch.max(outputs,0)
    # file1.write(str(outputs.detach().numpy()) + '\n')
    file2.write(str(tokenizedtest.index.values[idx]) + '\t' 
                + str(categories[idx1]) + '\t' + str(outputs.detach().numpy()) + '\n')
    
# file1.close()
file2.close()

In [ ]:
tokenizedtest.index.is_unique

In [ ]:
preds.shape

# Concatenate data with Topics and Scores

In [16]:
# Load data

data = pd.read_csv('../data/final_merged_tweets.csv', index_col=0)
data['publish_date'] = pd.to_datetime(data['publish_date'])
data = data[data.publish_date > '2015-07-01']

/anaconda3/envs/ada/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,8,12,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/ada/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
data.publish_date.max()

In [17]:
predsAugJan = pd.read_csv('./predictions/predictions_aug-jan_2015.csv',
                          header=None, names=['topic', 'topic_scores'], sep='\t')
predsJanMar = pd.read_csv('./predictions/predictions_jan-mar.csv',
                          header=None, names=['topic', 'topic_scores'], sep='\t')
predsMarJun = pd.read_csv('./predictions/predictions_mar-jun.csv',
                          header=None, names=['topic', 'topic_scores'], sep='\t')
predsJunDec = pd.read_csv('./predictions/predictions_jun-dec.csv',
                          header=None, names=['topic', 'topic_scores'], sep='\t')
len(predsAugJan) + len(predsJanMar) + len(predsMarJun) + len(predsJunDec)

preds = pd.concat([predsAugJan, predsJanMar, predsMarJun, predsJunDec])


# Strip shit

In [19]:
aug_jan = open('./predictions/oldtweetspredictions.csv','r') 
new_aug_jan = open('./predictions/oldtweetspredictions.txt','w') 

In [20]:
import re
for line in aug_jan:
    if re.search('[A-Z]', line):
        new_aug_jan.write('\n' + line.rstrip())
    else:
        new_aug_jan.write(line.rstrip())

In [21]:
new_aug_jan.close()

In [22]:
# test= pd.read_csv('./predictions/predictions_new_aug-jan_2015.txt',
#                   header=None, sep='\t',names=['topic', 'topic_scores'], index_col=0)


# predsAugJan = pd.read_csv('./predictions/predictions_new_aug-jan_2015.txt',
#                           header=None, names=['topic', 'topic_scores'], sep='\t')
# predsJanMar = pd.read_csv('./predictions/predictions_new_jan-mar.txt',
#                           header=None, names=['topic', 'topic_scores'], sep='\t')
# predsMarJun = pd.read_csv('./predictions/predictions_new_mar-jun.txt',
#                           header=None, names=['topic', 'topic_scores'], sep='\t')
# predsJunDec = pd.read_csv('./predictions/predictions_new_jun-dec.txt',
#                           header=None, names=['topic', 'topic_scores'], sep='\t')

# preds = pd.concat([predsAugJan, predsJanMar, predsMarJun, predsJunDec])



predsold = pd.read_csv('./predictions/oldtweetspredictions.txt',
                          header=None, names=['topic', 'topic_scores'], sep='\t')

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.options.display.max_colwidth = 1000
preds.head(100)

In [ ]:
new_data = data[data.index.isin(preds.index)]

In [ ]:
new_topics = preds[preds.index.isin(new_data.index)]

In [ ]:
new_df = pd.concat([new_data, new_topics], join='inner', axis=1)

In [ ]:
new_df.publish_date.min()

In [ ]:
new_df.to_csv('FINAL_DF_WITH_TOPICS.csv')

In [ ]:
new_df.publish_date.min()

In [ ]:
new_df.publish_date.max()

In [ ]:
# Load data

data = pd.read_csv('../data/final_merged_tweets.csv', index_col=0)
data['publish_date'] = pd.to_datetime(data['publish_date'])
data = data[(data.publish_date > '2016-09-01') & (data.publish_date < '2016-11-10')]

In [ ]:
data['Topic'] = preds.values

In [ ]:
data.to_csv('tweet_dataset_2_months.csv')

In [ ]:
data.head()

In [ ]:
data = pd.read_csv('tweet_dataset_2_months.csv')
data['publish_date'] = pd.to_datetime(data['publish_date'])

In [ ]:
data_filtered = data[['publish_date', 'followers', 'author', 'like_count', 'Topic']]

In [ ]:
topics = data_filtered.Topic.value_counts().iloc[0:13].index.tolist()
topics

In [ ]:


x = len(trump)